In [1]:
clc; clear; close all;

% --------------------------------------------------------------
% Step 1: Load inbuilt MATLAB color image
% --------------------------------------------------------------
img = imread('peppers.png');
img = im2double(img);
[rows, cols, ch] = size(img);

% Reshape image into Nx3 array (pixels)
pixels = reshape(img, [], 3);

% --------------------------------------------------------------
% Step 2: Set Mean Shift parameters
% --------------------------------------------------------------
hs = 0.1;    % Color bandwidth (smaller = more clusters)
max_iters = 15;
threshold = 1e-3;

N = size(pixels,1);

% --------------------------------------------------------------
% Step 3: Mean Shift Mode-Seeking
% --------------------------------------------------------------
modes = pixels;     % Start each pixel at its own initial location

fprintf('Running Mean Shift Clustering...\n');

for iter = 1:max_iters
    fprintf("Iteration %d/%d\n", iter, max_iters);

    for i = 1:N
        % Compute distance from pixel i to all pixels
        dist = sqrt(sum((pixels - modes(i,:)).^2, 2));

        % Find pixels within bandwidth (hs)
        neighbors = pixels(dist < hs, :);

        % Compute mean of neighbors
        if ~isempty(neighbors)
            new_point = mean(neighbors, 1);
        else
            new_point = modes(i,:);
        end

        % Update shift
        shift = norm(new_point - modes(i,:));
        modes(i,:) = new_point;

        % Check convergence
        if shift < threshold
            continue;
        end
    end
end

% --------------------------------------------------------------
% Step 4: Group points that converge to the same mode
% --------------------------------------------------------------
fprintf("Assigning clusters based on converged modes...\n");

cluster_labels = zeros(N,1);
cluster_count = 0;
modes_final = [];

for i = 1:N
    assigned = false;
    for j = 1:cluster_count
        if norm(modes(i,:) - modes_final(j,:)) < 0.02  % threshold for merging
            cluster_labels(i) = j;
            assigned = true;
            break;
        end
    end

    if ~assigned
        cluster_count = cluster_count + 1;
        modes_final(cluster_count,:) = modes(i,:);
        cluster_labels(i) = cluster_count;
    end
end

fprintf("Total clusters found: %d\n", cluster_count);

% --------------------------------------------------------------
% Step 5: Reconstruct segmented image
% --------------------------------------------------------------
seg_img = zeros(size(pixels));

for k = 1:cluster_count
    seg_img(cluster_labels == k, :) = modes_final(k,:);
end

seg_img = reshape(seg_img, rows, cols, 3);

% --------------------------------------------------------------
% Step 6: Display Results
% --------------------------------------------------------------
figure;
subplot(1,2,1); imshow(img); title('Original Image');
subplot(1,2,2); imshow(seg_img); title('Mean Shift Segmented Image');